In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dados = pd.read_csv('./MICRODADOS_ENEM_2022.csv', sep=';', encoding='ISO-8859-1')
dados.dropna(inplace=True)
dados.reset_index(drop=True, inplace=True)

In [ ]:
target = dados.NU_NOTA_MT
nota_humanas = dados.NU_NOTA_CH
sexo = dados.TP_SEXO
cor_raca = dados.TP_COR_RACA
renda = dados.Q006
treineiro = dados.IN_TREINEIRO

In [ ]:
cor_raca = pd.Categorical(cor_raca)
renda = pd.Categorical(renda)
treineiro = pd.Categorical(treineiro)

cor_raca = cor_raca.rename_categories(['Não declarado', 'Branca', 'Preta', 'Parda', 'Amarela', 'Indígena'])
renda = renda.rename_categories([
    'Nenhuma Renda', 
    'Até R$ 1.212,00', 
    'De R$ 1.212,01 até R$ 1.818,00',
    'De R$ 1.818,01 até R$ 2.424,00',
    'De R$ 2.424,01 até R$ 3.030,00',
    'De R$ 3.030,01 até R$ 3.636,00',
    'De R$ 3.636,01 até R$ 4.848,00',
    'De R$ 4.848,01 até R$ 6.060,00',
    'De R$ 6.060,01 até R$ 7.272,00',
    'De R$ 7.272,01 até R$ 8.484,00',
    'De R$ 8.484,01 até R$ 9.696,00',
    'De R$ 9.696,01 até R$ 10.908,00',
    'De R$ 10.908,01 até R$ 12.120,00',
    'De R$ 12.120,01 até R$ 14.544,00',
    'De R$ 14.544,01 até R$ 18.180,00',
    'De R$ 18.180,01 até R$ 24.240,00',
    'Acima de R$ 24.240,00'
])
treineiro = treineiro.rename_categories(['Não Treineiro', 'Treineiro'])

dados['TP_COR_RACA'] = cor_raca
dados['Q006'] = renda
dados['IN_TREINEIRO'] = treineiro

### Comparando a variável target (Nota de matemática) com a feature (Sexo)

In [ ]:
nota_mat_masc = dados.loc[dados.TP_SEXO == 'M', 'NU_NOTA_MT']
nota_mat_fem = dados.loc[dados.TP_SEXO == 'F', 'NU_NOTA_MT']

amp = np.linspace(0, 1000, 21)
plt.figure(figsize=(12, 6))
# plt.subplot(211)
plt.hist(nota_mat_masc, bins=amp, edgecolor='black', density=True)
plt.title('Masculino')
plt.xticks(amp)
# plt.subplot(212)
plt.hist(nota_mat_fem, bins=amp, edgecolor='black', density=True, alpha=0.5)
plt.title('Feminino')
plt.xticks(amp)
plt.show()

In [ ]:
dados.groupby('TP_SEXO').describe()['NU_NOTA_MT'].T.apply(lambda x: x.round(2))

### Comparando a variável target (Nota de matemática) com a feature (Nota de humanas)

In [ ]:
target.corr(nota_humanas)

### Comparando a variável target (Nota de matemática) com a feature (Cor/Raça)

In [ ]:
amp = np.linspace(0, 1000, 21)
for category in cor_raca.categories:
    notas = dados.loc[dados.TP_COR_RACA == category, 'NU_NOTA_MT']
    plt.figure(figsize=(12, 6))
    plt.hist(notas, bins=amp, alpha=0.5, label=category, density=True, edgecolor='black')
    plt.legend()
    plt.xticks(amp)
    plt.show()


In [ ]:
dados.groupby('TP_COR_RACA').describe()['NU_NOTA_MT'].T.apply(lambda x: x.round(2))

### Comparando a variável target (Nota de matemática) com a feature (Treineiro)

In [ ]:
amp = np.linspace(0, 1000, 21)
plt.figure(figsize=(12, 6))
for category in treineiro.categories:
    notas = dados.loc[dados.IN_TREINEIRO == category, 'NU_NOTA_MT']
    plt.hist(notas, bins=amp, alpha=0.5, label=category, density=True, edgecolor='black')
    plt.legend()
    plt.xticks(amp)

plt.show()

In [ ]:
dados.groupby('IN_TREINEIRO').describe()['NU_NOTA_MT'].T.apply(lambda x: x.round(2))

### Comparando a variável target (Nota de matemática) com a feature (Renda)

In [ ]:
amp = np.linspace(0, 1000, 21)
for category in renda.categories:
    notas = dados.loc[dados.Q006 == category, 'NU_NOTA_MT']
    plt.figure(figsize=(12, 6))
    plt.hist(notas, bins=amp, alpha=0.5, label=category, density=True, edgecolor='black')
    plt.legend()
    plt.xticks(amp)
    plt.show()

In [ ]:
dados.groupby('Q006').describe()['NU_NOTA_MT'].T.apply(lambda x: x.round(2))